In [ ]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://pueblosoriginarios.com/lenguas/mayas.php'

response = requests.get(url)

if response.status_code == 200:

  soup = BeautifulSoup(response.text,'html.parser')

  table = soup.find('table',{'id':'diccionario'})

  rows = table.find_all('tr')

  diccionario_data = []

  for row in rows[1:]:
    cols = row.find_all('td')

    if len(cols) == 2:
      maya_word = cols[0].text.strip()
      spanish_word = cols[1].text.strip()
      diccionario_data.append({'Maya':maya_word, 'Español':spanish_word})

  json_data = json.dumps(diccionario_data, indent=4, ensure_ascii=False)

  with open('diccionario_maya.json', 'w', encoding='utf-8') as file:
    file.write(json_data)

  print("Datos extraidos y guardados en diccionario_maya.json")

else:

  print("error al conectarse a la web")



In [ ]:
import pandas as pd

# Cargar los datos desde archivo json
with open('diccionario_maya.json','r', encoding='utf-8') as file:
  diccionario_data = json.load(file)

# Crear un dataframe
df = pd.DataFrame(diccionario_data)

# Crear el excel a partir del DataFrame
df.to_excel('diccionario_maya.xlsx', index=False)

print("Datos guardados en el diccionario_maya.xlsx")

In [ ]:
import pandas as pd
import json

# Cargar los datos desde archivo json
with open('diccionario_maya.json','r', encoding='utf-8') as file:
  diccionario_data = json.load(file)

# Crear un dataframe
df = pd.DataFrame(diccionario_data)

# 1. Estadisticas descriptiva

descripcion_maya = df['Maya'].describe()
descripcion_espanol = df['Español'].describe()

print("Estadistica descriptiva de las palabras Maya")
print(descripcion_maya)

print("Estadistica descriptiva de las palabras Español")
print(descripcion_espanol)

#2. filtrar datos

palabras_con_a = df[df['Maya'].str.startswith('a',na=False)]

print("Palabras que comienzan con 'a' : ")
print(palabras_con_a)

#3. ordenar datos

df_ordenado = df.sort_values(by='Maya', key=lambda x: x.str.len(),ascending=True)

print("Palabras Ordenadas :")
print(df_ordenado)

#4. Contar duplicados

duplicados_maya = df[df.duplicated(subset='Maya', keep=False)]

print("Palabras Duplicadas en Maya")
print(duplicados_maya)


In [ ]:
!pip install mysql.connector

In [ ]:
import mysql.connector
import json

#CONECTAR A LA BDATOS
conn = mysql.connector.connect(
    host="mysql-edisong.alwaysdata.net",
    user="edisong_us",
    password="pixel399",
    database="edisong_us"
)

#VARIABLE DE CONSULTA A LA BASE DEDATOS
cursor = conn.cursor()

#CARGAR EL ARCHIVO JSON
with open ('diccionario_maya.json', 'r', encoding = 'utf-8') as file:
  diccionario_data = json.load(file)

for palabra in diccionario_data:
  maya = palabra ['Maya']
  espanol = palabra['Español']
  insert_query = "INSERT INTO diccionario_json (maya,espanol) VALUES (%s,%s)"
  cursor.execute(insert_query, (maya,espanol))

conn.commit()
cursor.close()
conn.close()

In [ ]:
import mysql.connector
import pandas as pd

#CONECTAR A LA BDATOS
conn = mysql.connector.connect(
    host="mysql-edisong.alwaysdata.net",
    user="edisong_us",
    password="pixel399",
    database="edisong_us"
)

#VARIABLE DE CONSULTA A LA BASE DEDATOS
cursor = conn.cursor()

#CARGAR datos desde el archivo xlsx
df_excel = pd.read_excel ('diccionario_maya.xlsx')

for index, row in df_excel.iterrows ():
  maya = row ['Maya']
  espanol = row['Español']
  insert_query = "INSERT INTO diccionario_xlsx (maya,espanol) VALUES (%s,%s)"
  cursor.execute(insert_query, (maya,espanol))

conn.commit()
cursor.close()
conn.close()




In [ ]:
import mysql.connector
import pandas as pd

#CONECTAR A LA BDATOS
conn = mysql.connector.connect(
    host="mysql-edisong.alwaysdata.net",
    user="edisong_us",
    password="pixel399",
    database="edisong_us"
)

#VARIABLE DE CONSULTA A LA BASE DEDATOS
cursor = conn.cursor()

# Ejecuta una consulta SQL para seleccionar las columnas maya y espanol de la tabla diccionario_json
cursor.execute("SELECT maya, espanol FROM diccionario_json")

# recupera todos los resultados de la consulta ejecutada anteriormente.
resultados = cursor.fetchall()

# Itera sobre los resultados recuperados y Imprime los valores de maya y espanol.
for fila in resultados:
  maya, espanol = fila
  print(f"Maya: {maya}, ---------- Español: {espanol}")

# Cierra el cursor y la conexión a la base de datos una vez que se hayan completado todas las operaciones necesarias.
cursor.close()
conn.close()